In [3]:
%cd /mnt/c/Users/user/Desktop/과제/binfo1-work

/mnt/c/Users/user/Desktop/과제/binfo1-work


/root/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [6]:
!conda install -y subread

Retrieving notices: ...working... done
Channels:
 - conda-forge
 - bioconda
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.



In [7]:
!featureCounts -a gencode.vM27.annotation.gtf -o read-counts.txt *.bam


        ==========     _____ _    _ ____  _____  ______          _____  
        =====         / ____| |  | |  _ \|  __ \|  ____|   /\   |  __ \ 
          =====      | (___ | |  | | |_) | |__) | |__     /  \  | |  | |
            ====      \___ \| |  | |  _ <|  _  /|  __|   / /\ \ | |  | |
              ====    ____) | |__| | |_) | | \ \| |____ / ____ \| |__| |
        ==========   |_____/ \____/|____/|_|  \_\______/_/    \_\_____/
	  v2.0.6

//========================== featureCounts setting ===========================\\
||                                                                            ||
||             Input files : 10 BAM files                                     ||
||                                                                            ||
||                           CLIP-35L33G.bam                                  ||
||                           CLIP-let7d.bam                                   ||
||                           CLIP-let7f-1.bam                    

In [4]:
import pandas as pd
cnts = pd.read_csv('read-counts.txt', sep='\t', comment='#')
cnts.head()

,Geneid,Chr,Start,End,Strand,Length,CLIP-35L33G.bam,CLIP-let7d.bam,CLIP-let7f-1.bam,CLIP-let7g.bam,RNA-control.bam,RNA-siLin28a.bam,RNA-siLuc.bam,RPF-siLin28a.bam,RPF-siLuc.bam,filtered-RPF-siLuc.bam
0,ENSMUSG00000102693.2,chr1,3143476,3144545,+,1070,0,0,0,0,0,0,0,0,0,0
1,ENSMUSG00000064842.3,chr1,3172239,3172348,+,110,0,0,0,0,0,0,0,0,0,0
2,ENSMUSG00000051951.6,chr1;chr1;chr1;chr1;chr1;chr1;chr1,3276124;3276746;3283662;3283832;3284705;349192...,3277540;3277540;3285855;3286567;3287191;349212...,-;-;-;-;-;-;-,6094,4,0,0,0,1,1,1,0,0,0
3,ENSMUSG00000102851.2,chr1,3322980,3323459,+,480,3,0,0,0,0,0,0,0,0,0
4,ENSMUSG00000103377.2,chr1,3435954,3438772,-,2819,0,0,0,0,0,0,0,0,0,0


In [5]:
cnts['clip_enrichment'] = cnts['CLIP-35L33G.bam'] / cnts['RNA-control.bam']
cnts['rden_change'] = (cnts['RPF-siLin28a.bam'] / cnts['RNA-siLin28a.bam']) / (cnts['RPF-siLuc.bam'] / cnts['RNA-siLuc.bam'])
cnts.head()

,Geneid,Chr,Start,End,Strand,Length,CLIP-35L33G.bam,CLIP-let7d.bam,CLIP-let7f-1.bam,CLIP-let7g.bam,RNA-control.bam,RNA-siLin28a.bam,RNA-siLuc.bam,RPF-siLin28a.bam,RPF-siLuc.bam,filtered-RPF-siLuc.bam,clip_enrichment,rden_change
0,ENSMUSG00000102693.2,chr1,3143476,3144545,+,1070,0,0,0,0,0,0,0,0,0,0,NaN,NaN
1,ENSMUSG00000064842.3,chr1,3172239,3172348,+,110,0,0,0,0,0,0,0,0,0,0,NaN,NaN
2,ENSMUSG00000051951.6,chr1;chr1;chr1;chr1;chr1;chr1;chr1,3276124;3276746;3283662;3283832;3284705;349192...,3277540;3277540;3285855;3286567;3287191;349212...,-;-;-;-;-;-;-,6094,4,0,0,0,1,1,1,0,0,0,4.0,NaN
3,ENSMUSG00000102851.2,chr1,3322980,3323459,+,480,3,0,0,0,0,0,0,0,0,0,inf,NaN
4,ENSMUSG00000103377.2,chr1,3435954,3438772,-,2819,0,0,0,0,0,0,0,0,0,0,NaN,NaN


In [10]:
#GOterm 찾는 것 부터 내가 수행
!pip install biopython
!pip install bioservices

In [68]:
'''
from bioservices import Ensembl
#너무 오래걸려서 폐기
def get_GO_term(Gene_id):
    ensembl = Ensembl()
    go_terms = []
    try:
        gene = ensembl.lookup(id=Gene_id, expand=True)
        if "GO" in gene:
            for go in gene["GO"]:
                go_terms.append((go["id"], go["term"]))
    except Exception as e:
        print(f'Error for {Gene_id}: {str(e)}')
    return go_terms
    '''


'\nfrom bioservices import Ensembl\n#너무 오래걸려서 폐기\ndef get_GO_term(Gene_id):\n    ensembl = Ensembl()\n    go_terms = []\n    try:\n        gene = ensembl.lookup(id=Gene_id, expand=True)\n        if "GO" in gene:\n            for go in gene["GO"]:\n                go_terms.append((go["id"], go["term"]))\n    except Exception as e:\n        print(f\'Error for {Gene_id}: {str(e)}\')\n    return go_terms\n    '

In [12]:
!pip install biomart
!pip install concurrent

ERROR: Could not find a version that satisfies the requirement concurrent (from versions: none)
ERROR: No matching distribution found for concurrent


In [67]:
'''
#이것도 왠지 모르겠지만 잘 안돌아가서 폐기
from biomart import BiomartServer

def get_go_terms(gene_id):
    # Biomart 서버에 연결
    server = BiomartServer("http://www.ensembl.org/biomart")
    
    # Ensembl 데이터베이스에 액세스하는 데이터셋 선택
    dataset = server.datasets['mmusculus_gene_ensembl']
    
    # 필요한 필드 선택
    fields = ['ensembl_gene_id', 'go_id', 'name_1006']
    
    # 필터 설정: 유전자 ID로 필터링
    filters = {'ensembl_gene_id': gene_id}
    
    # Biomart를 사용하여 데이터 조회
    response = dataset.search({'filters': filters, 'attributes': fields})
    
    # 결과 처리
    go_terms = []
    for row in response.iter_lines():
        row_data = row.decode('utf-8').split('\t')
        go_terms.append((row_data[1], row_data[2]))  # GO ID와 GO 용어를 가져옴
    
    return go_terms
'''

'\n#이것도 왠지 모르겠지만 잘 안돌아가서 폐기\nfrom biomart import BiomartServer\n\ndef get_go_terms(gene_id):\n    # Biomart 서버에 연결\n    server = BiomartServer("http://www.ensembl.org/biomart")\n    \n    # Ensembl 데이터베이스에 액세스하는 데이터셋 선택\n    dataset = server.datasets[\'mmusculus_gene_ensembl\']\n    \n    # 필요한 필드 선택\n    fields = [\'ensembl_gene_id\', \'go_id\', \'name_1006\']\n    \n    # 필터 설정: 유전자 ID로 필터링\n    filters = {\'ensembl_gene_id\': gene_id}\n    \n    # Biomart를 사용하여 데이터 조회\n    response = dataset.search({\'filters\': filters, \'attributes\': fields})\n    \n    # 결과 처리\n    go_terms = []\n    for row in response.iter_lines():\n        row_data = row.decode(\'utf-8\').split(\'\t\')\n        go_terms.append((row_data[1], row_data[2]))  # GO ID와 GO 용어를 가져옴\n    \n    return go_terms\n'

In [6]:
!pip install Bio

In [7]:
from Bio import Entrez

Entrez.email = "your.email@example.com"

def fetch_go_id(go_id):
    handle = Entrez.esearch(db="gene", term=go_id)
    record = Entrez.read(handle)
    handle.close()
    return record

old_go_id = "GO:0005634"
result = fetch_go_id(old_go_id)
print(result)

{'Count': '6810', 'RetMax': '20', 'RetStart': '0', 'IdList': ['100259532', '3503598', '3503593', '3503543', '3503505', '3503443', '3503435', '3503430', '3503396', '3503354', '3503309', '3503305', '3503298', '3503243', '3503139', '3503121', '3503104', '3503007', '3502980', '3502965'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'GO[All Fields]', 'Field': 'All Fields', 'Count': '25065017', 'Explode': 'N'}, {'Term': '0005634[All Fields]', 'Field': 'All Fields', 'Count': '6812', 'Explode': 'N'}, 'AND', 'GROUP'], 'QueryTranslation': 'GO[All Fields] AND 0005634[All Fields]'}


In [64]:
from biomart import BiomartServer

# BioMart 서버에 연결
server = BiomartServer("http://www.ensembl.org/biomart")
dataset = server.datasets['mmusculus_gene_ensembl']

# 사용 가능한 필터 목록 가져오기
filters = dataset.filters

print(filters)


{'chromosome_name': 'Chromosome/scaffold name' (type: text, values: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, GL456210.1, GL456211.1, GL456212.1, GL456219.1, GL456221.1, GL456239.1, GL456354.1, GL456372.1, GL456381.1, GL456385.1, JH584296.1, JH584297.1, JH584298.1, JH584299.1, JH584303.1, JH584304.1, MT, X, Y]), 'start': 'Start' (type: text, values: []), 'end': 'End' (type: text, values: []), 'band_start': 'Band Start' (type: text, values: []), 'band_end': 'Band End' (type: text, values: []), 'strand': 'Strand' (type: text, values: []), 'chromosomal_region': 'e.g. 1:100:10000:-1, 1:100000:200000:1' (type: text, values: []), 'with_biogrid': 'With BioGRID Interaction data, The General Repository for Interaction Datasets ID(s)' (type: boolean_list, values: [True, False, excluded, included, only]), 'with_ccds': 'With CCDS ID(s)' (type: boolean_list, values: [True, False, excluded, included, only]), 'with_chembl': 'With ChEMBL ID(s)' (type: boolean_list, values: [T

In [8]:
'''
GPT가 그동안 mouse cell로 가동시킴

from biomart import BiomartServer
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_go_terms_batch(gene_ids, go_terms_filter):
    server = BiomartServer("http://www.ensembl.org/biomart")
    dataset = server.datasets['mmusculus_gene_ensembl']
    fields = ['ensembl_gene_id', 'go_id', 'name_1006']
    filters = {
        'ensembl_gene_id': gene_ids,
        'go': go_terms_filter
    }
    
    response = dataset.search({'filters': filters, 'attributes': fields})
    
    go_terms = []
    for row in response.iter_lines():
        row_data = row.decode('utf-8').split('\t')
        go_terms.append((row_data[0]))#, row_data[1], row_data[2]))  # ensembl_gene_id, go_id, name_1006
    
    return go_terms

def get_go_terms_all(gene_id_list, go_terms_filter, batch_size=100):
    go_terms = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
        for i in range(0, len(gene_id_list), batch_size):
            batch = gene_id_list[i:i + batch_size]
            futures.append(executor.submit(get_go_terms_batch, batch, go_terms_filter))

        print(futures)
        for future in as_completed(futures):
            try:
                result = future.result()
                go_terms.extend(result)
            except Exception as e:
                print(f"Error: {e}")
    
    return go_terms

# Example usage
gene_id_list = ['ENSMUSG00000064341', 'ENSMUSG00000064342', ...]  # 총 22000개의 gene ID 리스트
specific_go_terms = ['GO:0003677', 'GO:0008150']  # 조회하고 싶은 특정 GO term
go_terms = get_go_terms_all(gene_id_list, specific_go_terms, batch_size=100)

print(go_terms)
'''

"\n# Example usage\ngene_id_list = ['ENSMUSG00000064341', 'ENSMUSG00000064342', ...]  # 총 22000개의 gene ID 리스트\nspecific_go_terms = ['GO:0003677', 'GO:0008150']  # 조회하고 싶은 특정 GO term\ngo_terms = get_go_terms_all(gene_id_list, specific_go_terms, batch_size=100)\n\nprint(go_terms)\n"

In [37]:
import gzip

# GZIP 파일 읽기 및 필터링
with gzip.open("goa_mouse.gaf.gz", 'rt') as f:
    # GAF 파일을 데이터프레임으로 읽기
    mouse_data = pd.read_csv(f, sep='\t', comment='!', header=None)

# 열 이름 설정
mouse_data.columns = [
    'DB', 'DB_Object_ID', 'DB_Object_Symbol', 'Qualifier', 'GO_ID',
    'DB:Reference', 'Evidence_Code', 'With_From', 'Aspect', 'DB_Object_Name',
    'DB_Object_Synonym', 'DB_Object_Type', 'Taxon', 'Date', 'Assigned_By',
    'Annotation_Extension', 'Gene_Product_Form_ID'
]

# 데이터 출력 (예: 상위 5개 행)
print(mouse_data.head())

# 필요한 경우 데이터 저장
mouse_data.to_csv('mouse_go_annotations.gaf', sep='\t', index=False)

          DB DB_Object_ID DB_Object_Symbol     Qualifier       GO_ID  \
0  UniProtKB   A0A075B5I2            Trbv4   involved_in  GO:0007166   
1  UniProtKB   A0A075B5I2            Trbv4  is_active_in  GO:0005886   
2  UniProtKB   A0A075B5J4            Trbc2       part_of  GO:0042105   
3  UniProtKB   A0A075B5J6          Gm20730   involved_in  GO:0006955   
4  UniProtKB   A0A075B5J6          Gm20730  is_active_in  GO:0005615   

     DB:Reference Evidence_Code  \
0  GO_REF:0000033           IBA   
1  GO_REF:0000033           IBA   
2  GO_REF:0000033           IBA   
3  GO_REF:0000033           IBA   
4  GO_REF:0000033           IBA   

                                           With_From Aspect  \
0                 MGI:MGI:98608|PANTHER:PTN000588091      P   
1  MGI:MGI:98596|PANTHER:PTN000588091|UniProtKB:A...      C   
2              PANTHER:PTN001511926|UniProtKB:A0A5B9      C   
3  MGI:MGI:98936|PANTHER:PTN000587099|UniProtKB:P...      P   
4              PANTHER:PTN000587099|UniPr

In [12]:
cnts['Geneid'] = cnts['Geneid'].apply(lambda x: x.split('.')[0])

In [13]:
cnts = cnts.dropna(subset=['clip_enrichment'])
cnts = cnts.dropna(subset=['rden_change'])
cnts.head()

,Geneid,Chr,Start,End,Strand,Length,CLIP-35L33G.bam,CLIP-let7d.bam,CLIP-let7f-1.bam,CLIP-let7g.bam,RNA-control.bam,RNA-siLin28a.bam,RNA-siLuc.bam,RPF-siLin28a.bam,RPF-siLuc.bam,filtered-RPF-siLuc.bam,clip_enrichment,rden_change
15,ENSMUSG00000102343,chr1;chr1;chr1;chr1;chr1,3975962;4054448;4055383;4055383;4056370,3976357;4054521;4056207;4055574;4056438,-;-;-;-;-,1364,67,0,0,0,31,207,157,1,2,0,2.161290,0.379227
19,ENSMUSG00000025902,chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;chr1;c...,4561154;4561473;4561613;4561936;4562680;456268...,4562891;4562886;4562891;4562891;4563827;456289...,-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;-;...,4772,6,0,0,0,1,8,5,2,3,0,6.000000,0.416667
21,ENSMUSG00000102269,chr1;chr1,4593128;4594669,4593826;4596960,+;+,2991,8,0,0,0,73,28,11,2,9,9,0.109589,0.087302
26,ENSMUSG00000098104,chr1,4758157,4759626,-,1470,29,0,0,0,63,26,15,2,7,0,0.460317,0.164835
30,ENSMUSG00000103922,chr1,4841354,4842422,+,1069,183,0,0,0,95,93,100,42,75,74,1.926316,0.602151


In [14]:
gene_IDs = cnts['Geneid'].tolist()
#print(gene_IDs)
print(len(gene_IDs))

21231


In [41]:
file_path = 'mouse_go_annotations.gaf'

# GAF 파일을 데이터프레임으로 읽기
gaf_data = pd.read_csv(file_path, sep='\t', comment='!', header=None)

# 열 이름 설정
gaf_data.columns = [
    'DB', 'DB_Object_ID', 'DB_Object_Symbol', 'Qualifier', 'GO_ID',
    'DB:Reference', 'Evidence_Code', 'With_From', 'Aspect', 'DB_Object_Name',
    'DB_Object_Synonym', 'DB_Object_Type', 'Taxon', 'Date', 'Assigned_By',
    'Annotation_Extension', 'Gene_Product_Form_ID'
]

def find_gene_id(go_id):
    # 입력된 gene ID에 해당하는 행 찾기
    matching_rows = gaf_data[gaf_data['GO_ID'] == go_id]
    
    if len(matching_rows) == 0:
        #print("Gene ID not found.")
        return []
    
    # 해당하는 행에서 GO ID 추출
    gene_ids = matching_rows['DB_Object_ID'].tolist()
    print(gene_ids)
    return gene_ids

# Gene ID 입력 받기
#gene_id = input("Enter Gene ID: ")


/tmp/ipykernel_20231/4009362820.py:4: DtypeWarning: Columns (13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  gaf_data = pd.read_csv(file_path, sep='\t', comment='!', header=None)


In [43]:
!pip install requests

In [45]:
import requests

def query_biomart(ensembl_gene_ids):
    # Define the BioMart query
    xml_query = f"""
    <!DOCTYPE Query>
    <Query  virtualSchemaName = "default" formatter = "TSV" header = "1" uniqueRows = "1" count = "" datasetConfigVersion = "0.6" >
        <Dataset name = "mmusculus_gene_ensembl" interface = "default" >
            <Filter name = "ensembl_gene_id" value = "{','.join(ensembl_gene_ids)}"/>
            <Attribute name = "ensembl_gene_id" />
            <Attribute name = "uniprotswissprot" />
            <Attribute name = "uniprotsptrembl" />
        </Dataset>
    </Query>
    """
    
    # Send the query to the BioMart service
    response = requests.post('https://www.ensembl.org/biomart/martservice', data={'query': xml_query})
    response.raise_for_status()
    
    # Parse the response
    lines = response.text.strip().split('\n')
    headers = lines[0].split('\t')
    data = [line.split('\t') for line in lines[1:]]
    
    return headers, data

def get_uniprot_ids_for_genes(ensembl_gene_ids):
    headers, data = query_biomart(ensembl_gene_ids)
    
    gene_to_uniprot = {}
    for row in data:
        ensembl_gene_id = row[0]
        uniprot_ids = row[1:]  # Both Swiss-Prot and TrEMBL IDs
        uniprot_ids = [uid for uid in uniprot_ids if uid]  # Remove empty strings
        
        if ensembl_gene_id not in gene_to_uniprot:
            gene_to_uniprot[ensembl_gene_id] = set()
        
        gene_to_uniprot[ensembl_gene_id].update(uniprot_ids)
    
    # Convert sets to lists
    for gene_id in gene_to_uniprot:
        gene_to_uniprot[gene_id] = list(gene_to_uniprot[gene_id])
    
    return gene_to_uniprot

# Example Ensembl Gene IDs
ensembl_gene_ids = ["ENSMUSG00000025902"]

# Get UniProt IDs for the given Ensembl Gene IDs
gene_to_uniprot = get_uniprot_ids_for_genes(ensembl_gene_ids)

# Print the results
for gene_id, uniprot_ids in gene_to_uniprot.items():
    print(f"Gene ID: {gene_id}, UniProt IDs: {uniprot_ids}")

Gene ID: ENSMUSG00000025902, UniProt IDs: ['A0A0A6YXS3', 'A0A0A6YWS4', 'Q61473', 'A0A0A6YXZ2', 'A0A0A6YXV3']


In [42]:
nucleus = []
#find_go_id('ENSMUSG00000025902')

find_gene_id('GO:0005737')

'''
for i in range(len(gene_IDs)):
    find_go_id(gene_IDs[i])
    if find_go_id(gene_IDs[i]) != []:
        print(gene_IDs[i])
        '''

['A0A087WPF7', 'A0A087WRH8', 'A0A087WRK1', 'A0A087WRK1', 'A0A088MLT8', 'A0A0A6YVU8', 'A0A0A6YW67', 'A0A0A6YWS7', 'A0A0B4J1F4', 'A0A0G2JEB6', 'A0A0G2JFS9', 'A0A0J9YU00', 'A0A0J9YU01', 'A0A0J9YU88', 'A0A0J9YUM1', 'A0A0J9YUV1', 'A0A0J9YVD0', 'A0A0N4SUH6', 'A0A140LI04', 'A0A140LI67', 'A0A140LJ87', 'A0A140T8U1', 'A0A1B0GR71', 'A0A1B0GRF0', 'A0A1B0GRG8', 'A0A1B0GRV7', 'A0A1B0GSB0', 'A0A1B0GSG7', 'A0A1B0GSW2', 'A0A1B0GT24', 'A0A1B0GT45', 'A0A1D5RLD8', 'A0A1D5RLF0', 'A0A1D5RM54', 'A0A1D5RMN0', 'A0A1W2P884', 'A0A1Y7VJE9', 'A0A1Y7VK80', 'A0A1Y7VKT9', 'A0A1Y7VLT7', 'A0A1Y7VNG9', 'A0A286YCX6', 'A0A286YEC0', 'A0A494B9V8', 'A0A571BEB1', 'A0A571BEG0', 'A0A571BF02', 'A0A5F8MPH2', 'A0A6I8MWX0', 'A0A7N9VR94', 'A0A7N9VSG0', 'A0AUP1', 'A0JNT9', 'A1E960', 'A1E960', 'A1E960', 'A1E960', 'A1IGU4', 'A1Y9I9', 'A1Z198', 'A2A484', 'A2A5Z6', 'A2A6M5', 'A2A6Q5', 'A2A7Q9', 'A2A825', 'A2A870', 'A2A8M8', 'A2A8N0', 'A2A8N2', 'A2A8R0', 'A2A935', 'A2A958', 'A2A9F4', 'A2A9R3', 'A2A9T0', 'A2AA28', 'A2AA28', 'A2AAY5', 'A2AA

'\nfor i in range(len(gene_IDs)):\n    find_go_id(gene_IDs[i])\n    if find_go_id(gene_IDs[i]) != []:\n        print(gene_IDs[i])\n        '

In [40]:
GO_terms = []
for i in range(3):
    a = get_go_terms(gene_IDs[i])
    GO_terms.append(a)

print(GO_terms)

[[('', '')], [('GO:0010467', 'gene expression'), ('GO:1990830', 'cellular response to leukemia inhibitory factor'), ('GO:0005634', 'nucleus'), ('GO:0043565', 'sequence-specific DNA binding'), ('GO:0010628', 'positive regulation of gene expression'), ('GO:0045944', 'positive regulation of transcription by RNA polymerase II'), ('GO:0005515', 'protein binding'), ('GO:0007283', 'spermatogenesis'), ('GO:2000738', 'positive regulation of stem cell differentiation'), ('GO:0048863', 'stem cell differentiation'), ('GO:0001525', 'angiogenesis'), ('GO:0061009', 'common bile duct development'), ('GO:0048568', 'embryonic organ development'), ('GO:0000976', 'transcription cis-regulatory region binding'), ('GO:0003677', 'DNA binding'), ('GO:0000978', 'RNA polymerase II cis-regulatory region sequence-specific DNA binding'), ('GO:0016055', 'Wnt signaling pathway'), ('GO:0001228', 'DNA-binding transcription activator activity, RNA polymerase II-specific'), ('GO:0006355', 'regulation of DNA-templated tra

In [103]:
target_GO_terms = ['GO:0010467']
go_terms = get_go_terms_all(gene_IDs[1], target_GO_terms, batch_size=100)
print(go_terms)

[<Future at 0x7f95087c8820 state=running>]
['ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00000025902', 'ENSMUSG00

In [65]:


go_terms = get_go_terms_all(gene_IDs[1], target_GO_terms, batch_size=100)
go_terms = get_go_terms_batch(gene_IDs[1], target_GO_terms)

print(go_terms)


[<Future at 0x7f955ca44e20 state=running>]
[('ENSMUSG00000025902', 'GO:0010467', 'gene expression'), ('ENSMUSG00000025902', 'GO:1990830', 'cellular response to leukemia inhibitory factor'), ('ENSMUSG00000025902', 'GO:0005634', 'nucleus'), ('ENSMUSG00000025902', 'GO:0043565', 'sequence-specific DNA binding'), ('ENSMUSG00000025902', 'GO:0010628', 'positive regulation of gene expression'), ('ENSMUSG00000025902', 'GO:0045944', 'positive regulation of transcription by RNA polymerase II'), ('ENSMUSG00000025902', 'GO:0005515', 'protein binding'), ('ENSMUSG00000025902', 'GO:0007283', 'spermatogenesis'), ('ENSMUSG00000025902', 'GO:2000738', 'positive regulation of stem cell differentiation'), ('ENSMUSG00000025902', 'GO:0048863', 'stem cell differentiation'), ('ENSMUSG00000025902', 'GO:0001525', 'angiogenesis'), ('ENSMUSG00000025902', 'GO:0061009', 'common bile duct development'), ('ENSMUSG00000025902', 'GO:0048568', 'embryonic organ development'), ('ENSMUSG00000025902', 'GO:0000976', 'transcrip

In [ ]:
'''
시간 오래걸려서 폐기 - 심각함 1300분 넘게 소요
GO_terms = []
for i in range(len(gene_IDs)):
    a = get_go_terms(gene_IDs[i])
    GO_terms.append(a)

print(GO_terms)
'''

In [49]:
#2만개 데이터 처리 오래걸려서 중복있나 조회
duplicates = cnts.duplicated(subset=['Geneid'])
print(cnts[duplicates])

Empty DataFrame
Columns: [Geneid, Chr, Start, End, Strand, Length, CLIP-35L33G.bam, CLIP-let7d.bam, CLIP-let7f-1.bam, CLIP-let7g.bam, RNA-control.bam, RNA-siLin28a.bam, RNA-siLuc.bam, RPF-siLin28a.bam, RPF-siLuc.bam, filtered-RPF-siLuc.bam, clip_enrichment, rden_change]
Index: []
